In [1]:
import json
from elasticsearch import Elasticsearch

es = Elasticsearch()
print (json.dumps(es.info(),indent=2))

{
  "tagline": "You Know, for Search",
  "cluster_name": "TwitterDB",
  "name": "Machine1",
  "version": {
    "build_timestamp": "2014-12-16T14:11:12Z",
    "lucene_version": "4.10.2",
    "number": "1.4.2",
    "build_snapshot": false,
    "build_hash": "927caff6f05403e936c20bf4529f144f0c89fd8c"
  },
  "status": 200
}


In [2]:
help(Elasticsearch)

Help on class Elasticsearch in module elasticsearch.client:

class Elasticsearch(builtins.object)
 |  Elasticsearch low-level client. Provides a straightforward mapping from
 |  Python to ES REST endpoints.
 |  
 |  The instance has attributes ``cat``, ``cluster``, ``indices``, ``ingest``,
 |  ``nodes``, ``snapshot`` and ``tasks`` that provide access to instances of
 |  :class:`~elasticsearch.client.CatClient`,
 |  :class:`~elasticsearch.client.ClusterClient`,
 |  :class:`~elasticsearch.client.IndicesClient`,
 |  :class:`~elasticsearch.client.IngestClient`,
 |  :class:`~elasticsearch.client.NodesClient`,
 |  :class:`~elasticsearch.client.SnapshotClient` and
 |  :class:`~elasticsearch.client.TasksClient` respectively. This is the
 |  preferred (and only supported) way to get access to those classes and their
 |  methods.
 |  
 |  You can specify your own connection class which should be used by providing
 |  the ``connection_class`` parameter::
 |  
 |      # create connection to localh

In [3]:
#uid對應screen_name
response = es.search(
    index="twitter2",
    doc_type="user",
    body={
            "query": {
                    "match_all": {}
            },
            "size":200,
            "sort":{"uid":"asc"}
            
        }
)
#print (json.dumps(response,indent=2))
tuid_list=[]
tname_list=[]
for user in response['hits']['hits']:
   tuid_list.append(user['_source']['uid'])
   tname_list.append(user['_source']['screen_name'])
   print (user['_source']['uid'],user['_source']['screen_name'])

1102 davidu
48443 jeffbarr
658323 danphilpott
795029 samj
810641 oncee
813697 mckeay
1088411 gattaca
3107271 randybias
3247471 paulfroberts
4905551 masontech
7025212 jack_daniel
8236572 ryanaraine
8917142 dakami
9956632 mroesch
10351232 MikeWiacek
11443182 scamboy
11687162 andrewsmhay
11791512 gcluley
12219832 lmwalsh2112
12692452 anton_chuvakin
13089682 pdp
13275122 McGrewSecurity
13850552 raffaelmarty
14090906 WeldPond
14118608 dmkravets
14174808 jjx
14181505 jeremiahg
14244412 roblemos
14277681 BrianHonan
14293266 helpnetsecurity
14333690 DavidLinthicum
14415986 agent0x0
14666934 kevinmitnick
14669471 csoghoian
14780493 lennyzeltser
14803061 lptacek
14924745 thedarktangent
15040971 mattjay
15270591 smallbizprivacy
15447522 k8em0
15581551 mdowd
15589731 thierryzoller
15637093 RSnake
15637965 ah8r
15655289 carnal0wnage
15757132 edskoudis
15943215 BillBrenner70
16103879 sarapeters
16626073 SteveD3
16730420 SophosLabs
16935717 SecurityExpert
17226373 hal_pomeranz
17464103 ihackstuff
176

In [11]:
response = es.search(
    index="twitter2",
    body={
            "query": {
                "bool": {
                    "should":[
                        {"match":{"tweet.text":"vulnerability"}},
                        {"match":{"tweet.text":"cve"}},
                        {"match":{"tweet.text":"Exploit"}}
                    ],
                    "minimum_should_match": 1
                }
            },              
            "aggs": {
                "tweet": {
                    "terms": {
                        "field": "uid",
                        "size": 800000
                    }
                }
            }
    }
)
#print (json.dumps(response,indent=2))
twitters=[]
tweets=0
for tweet in response['aggregations']['tweet']['buckets']:
    for k in range(len(tuid_list)):
        if tuid_list[k]==tweet['key']:
            tname=tname_list[k]
#            print (tname)
    twitters.append(tweet['key'])
    tweets += tweet['doc_count']
#    print ("[Twitter]:",tname,' [Keyword Hints]:',tweet['doc_count'])
print()
print ("共計有多少Twitter參與這類關鍵字議題討論? Ans:",len(twitters))
print ("共計有多少Tweets? Ans:",tweets)

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.006s]



共計有多少Twitter參與這類關鍵字議題討論? Ans: 100
共計有多少Tweets? Ans: 4562


In [5]:
import time
from bokeh.charts import Bar, output_file, show
from bokeh.sampledata.autompg import autompg as df
from bokeh.layouts import row,widgetbox

In [10]:
response = es.search(
    index="twitter2",
    body={
            "query": {
                "bool": {
                    "should":[
                        {"match":{"tweet.text":"vulnerability"}},
                        {"match":{"tweet.text":"cve"}},
                        {"match":{"tweet.text":"Exploit"}}
                    ],
                    "minimum_should_match": 1
                }
            },
            "aggs": {
                "tweet": {
                    "date_histogram": {
                         "field": "created_at",
                         "interval": '1M'
                    },
                    "aggs": {
                        "tweet": {
                            "terms": {
                                "field": "uid",
                                "size": 200
                            }
                        }
                    }
                }
            }
        }
)
#print (json.dumps(response,indent=2))
month=[]
tweets=[]
#tcount=0
for tweet in response['aggregations']['tweet']['buckets']:
    mon=time.strftime('%Y-%m',(time.strptime(tweet['key_as_string'],'%a %b %d %H:%M:%S %z %Y')))
    print ("[Month]:",mon,' [Keyword Hints]:',tweet['doc_count'])
    month.append(mon)
    tweets.append(tweet['doc_count'])
#    tcount+=tweet['doc_count']
print()
#print (tcount)
bar1 = Bar(tweets,plot_width=1500)

output_file("bar1.html")
show(bar1)

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.022s]


[Month]: 2008-04  [Keyword Hints]: 1
[Month]: 2008-05  [Keyword Hints]: 4
[Month]: 2008-07  [Keyword Hints]: 3
[Month]: 2008-08  [Keyword Hints]: 1
[Month]: 2009-03  [Keyword Hints]: 3
[Month]: 2009-04  [Keyword Hints]: 1
[Month]: 2009-05  [Keyword Hints]: 2
[Month]: 2009-06  [Keyword Hints]: 2
[Month]: 2009-07  [Keyword Hints]: 6
[Month]: 2009-08  [Keyword Hints]: 1
[Month]: 2009-09  [Keyword Hints]: 3
[Month]: 2009-10  [Keyword Hints]: 3
[Month]: 2009-11  [Keyword Hints]: 9
[Month]: 2009-12  [Keyword Hints]: 4
[Month]: 2010-01  [Keyword Hints]: 9
[Month]: 2010-02  [Keyword Hints]: 6
[Month]: 2010-03  [Keyword Hints]: 8
[Month]: 2010-04  [Keyword Hints]: 5
[Month]: 2010-05  [Keyword Hints]: 8
[Month]: 2010-06  [Keyword Hints]: 11
[Month]: 2010-07  [Keyword Hints]: 7
[Month]: 2010-08  [Keyword Hints]: 16
[Month]: 2010-09  [Keyword Hints]: 19
[Month]: 2010-10  [Keyword Hints]: 6
[Month]: 2010-11  [Keyword Hints]: 10
[Month]: 2010-12  [Keyword Hints]: 7
[Month]: 2011-01  [Keyword Hints]:

INFO:bokeh.core.state:Session output file 'bar1.html' already exists, will be overwritten.


In [7]:
response = es.search(
    index="twitter2",
    body={
            "query": {
                "bool": {
                    "should":[
                        {"match":{"tweet.text":"vulnerability"}},
                        {"match":{"tweet.text":"cve"}},
                        {"match":{"tweet.text":"Exploit"}}
                    ],
                    "minimum_should_match": 1
                }
            },
            "aggs": {
                "tweet": {
                    "date_histogram": {
                         "field": "created_at",
                         "interval": '1M'
                    },
                    "aggs": {
                        "tweet": {
                            "terms": {
                                "field": "uid",
                                "size": 200
                            }
                        }
                    }
                }
            }
        }
)
#print (json.dumps(response,indent=2))
twitters=[]
num_twitters=[]
tweets=[]
#for tweet in response['aggregations']['tweet']['buckets']:
for l in range(len(response['aggregations']['tweet']['buckets'])):        
    for m in range(len(response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'])):
        twitters.append(response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'][m]['key'])
    mon=time.strftime('%Y-%m',(time.strptime(response['aggregations']['tweet']['buckets'][l]
                                             ['key_as_string'],'%a %b %d %H:%M:%S %z %Y')))
    print ("[Month]:",mon,"參與的Twitter數量 =",len(twitters),
           "每個月參Tweets數量 =",response['aggregations']['tweet']['buckets'][l]['doc_count'])
    num_twitters.append(len(twitters))
    tweets.append(response['aggregations']['tweet']['buckets'][l]['doc_count'])
    twitters=[]
bar1 = Bar(tweets,plot_width=1500)    
bar2 = Bar(num_twitters,plot_width=1500)

output_file("bar.html")
show(row(bar1,bar2))

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.018s]


[Month]: 2008-04 參與的Twitter數量 = 1 每個月參Tweets數量 = 1
[Month]: 2008-05 參與的Twitter數量 = 1 每個月參Tweets數量 = 4
[Month]: 2008-07 參與的Twitter數量 = 3 每個月參Tweets數量 = 3
[Month]: 2008-08 參與的Twitter數量 = 1 每個月參Tweets數量 = 1
[Month]: 2009-03 參與的Twitter數量 = 2 每個月參Tweets數量 = 3
[Month]: 2009-04 參與的Twitter數量 = 1 每個月參Tweets數量 = 1
[Month]: 2009-05 參與的Twitter數量 = 2 每個月參Tweets數量 = 2
[Month]: 2009-06 參與的Twitter數量 = 2 每個月參Tweets數量 = 2
[Month]: 2009-07 參與的Twitter數量 = 3 每個月參Tweets數量 = 6
[Month]: 2009-08 參與的Twitter數量 = 1 每個月參Tweets數量 = 1
[Month]: 2009-09 參與的Twitter數量 = 2 每個月參Tweets數量 = 3
[Month]: 2009-10 參與的Twitter數量 = 2 每個月參Tweets數量 = 3
[Month]: 2009-11 參與的Twitter數量 = 6 每個月參Tweets數量 = 9
[Month]: 2009-12 參與的Twitter數量 = 3 每個月參Tweets數量 = 4
[Month]: 2010-01 參與的Twitter數量 = 5 每個月參Tweets數量 = 9
[Month]: 2010-02 參與的Twitter數量 = 3 每個月參Tweets數量 = 6
[Month]: 2010-03 參與的Twitter數量 = 5 每個月參Tweets數量 = 8
[Month]: 2010-04 參與的Twitter數量 = 4 每個月參Tweets數量 = 5
[Month]: 2010-05 參與的Twitter數量 = 4 每個月參Tweets數量 = 8
[Month]: 2010-06 參與的Twitter數量 =

INFO:bokeh.core.state:Session output file 'bar.html' already exists, will be overwritten.


In [19]:
from bokeh.charts import Donut, show, output_notebook, vplot
from bokeh.charts.utils import df_from_json
import pandas as pd

response = es.search(
    index="twitter2",
    body={
            "query": {
                "bool": {
                    "should":[
                        {"match":{"tweet.text":"vulnerability"}},
                        {"match":{"tweet.text":"cve"}},
                        {"match":{"tweet.text":"Exploit"}}
                    ],
                    "minimum_should_match": 1
                }
            },
            "from": 0,
            "size": 800000
    }
)
#print (json.dumps(response,indent=2))
nurl=0
for tweet in response['hits']['hits']:
    if (tweet['_source']['entities']['urls']) == []:
        nurl=nurl+1
t_total=len(response['hits']['hits'])
print ('其中無URL的Tweets數=',nurl,',占比為',format(nurl/t_total, '.2%'))
print ('其中有URL的Tweets數=',t_total-nurl,',占比為',format((t_total-nurl)/t_total, '.2%'))



d=Donut(pd.Series([nurl , t_total-nurl] , index = ['無URL  '+format(nurl/t_total, '.2%'),'有URL  '+format((t_total-nurl)/t_total, '.2%')]))
show(d)


INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.274s]


其中無URL的Tweets數= 1033 ,占比為 22.64%
其中有URL的Tweets數= 3529 ,占比為 77.36%


In [82]:
response = es.search(
    index="twitter2",
    body={
            "query": {
                "bool": {
                    "must":{"match":{"tweet.text":"cve"}}
                }
            },
            "aggs": {
                "tweet": {
                    "date_histogram": {
                         "field": "created_at",
                         "interval": '1M'
                    },
                    "aggs": {
                        "tweet": {
                            "terms": {
                                "field": "uid",
                                "size": 200
                            }
                        }
                    }
                }
            },
            "from": 0,
            "size": 1000
    }
)
#print (json.dumps(response,indent=2))
twitters=[]
num_twitters=[]
tweets=[]
#for tweet in response['aggregations']['tweet']['buckets']:
for l in range(len(response['aggregations']['tweet']['buckets'])):
    mon=time.strftime('%Y-%m',(time.strptime(response['aggregations']['tweet']['buckets'][l]
                                             ['key_as_string'],'%a %b %d %H:%M:%S %z %Y')))
    print ("[Month]:",mon)
    for m in range(len(response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'])):
        twitters.append(response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'][m]['key'])
        for k in range(len(tuid_list)):
                if tuid_list[k]==response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'][m]['key']:
                    tname=tname_list[k]
        print (tname,":",response['aggregations']['tweet']['buckets'][l]['tweet']['buckets'][m]['doc_count'],'篇')
    mon=time.strftime('%Y-%m',(time.strptime(response['aggregations']['tweet']['buckets'][l]
                                             ['key_as_string'],'%a %b %d %H:%M:%S %z %Y')))
    #print ("本月參與的Twitter數量 =",len(twitters),
    #       "本月參Tweets數量 =",response['aggregations']['tweet']['buckets'][l]['doc_count'])
    tweets.append(response['aggregations']['tweet']['buckets'][l]['doc_count'])
    twitters=[]

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.020s]


[Month]: 2008-07
raffaelmarty : 1 篇
[Month]: 2009-11
agent0x0 : 1 篇
[Month]: 2010-01
beist : 1 篇
[Month]: 2010-03
cesarcer : 1 篇
[Month]: 2010-06
beist : 2 篇
[Month]: 2010-07
cesarcer : 1 篇
[Month]: 2010-09
beist : 1 篇
McAfee_Labs : 1 篇
chetwisniewski : 1 篇
[Month]: 2010-10
beist : 1 篇
[Month]: 2010-12
cesarcer : 1 篇
[Month]: 2011-01
cesarcer : 1 篇
[Month]: 2011-02
chetwisniewski : 1 篇
[Month]: 2011-04
chetwisniewski : 1 篇
cesarcer : 1 篇
[Month]: 2011-07
agent0x0 : 1 篇
[Month]: 2011-09
cesarcer : 1 篇
ryancbarnett : 1 篇
[Month]: 2011-10
cesarcer : 2 篇
[Month]: 2011-12
chetwisniewski : 1 篇
[Month]: 2012-01
ryancbarnett : 4 篇
mdowd : 1 篇
[Month]: 2012-02
mdowd : 1 篇
chetwisniewski : 1 篇
cesarcer : 1 篇
[Month]: 2012-03
ryanaraine : 1 篇
[Month]: 2012-04
ryanaraine : 1 篇
McAfee_Labs : 1 篇
[Month]: 2012-05
e_kaspersky : 1 篇
[Month]: 2012-06
ryanaraine : 2 篇
[Month]: 2012-07
chetwisniewski : 2 篇
[Month]: 2012-08
McAfee_Labs : 2 篇
chetwisniewski : 2 篇
[Month]: 2013-01
mdowd : 1 篇
justin_foster 

In [21]:
response = es.search(
    index="logstash*",
    body={
          "query": {
              "range": {
                  "@timestamp": {
                      "gt": "2016-10-01T15:39:11.000Z",
                      "lt": "2016-10-20T15:39:13.000Z"
                  }
              }
          },
          "from": 0,
          "size": 2
    }
)
print json.dumps(response,indent=2)

INFO:elasticsearch:GET http://localhost:9200/logstash*/_search [status:200 request:0.002s]


In [17]:
response = es.search(
    index="logstash*",
    body={
          "query": {
            "bool": {
              "must": [
                {
                  "range": {
                    "@timestamp": {
                      "gt": "2016-10-02T15:39:11.000Z",
                      "lt": "2016-10-20T16:39:13.000Z"
                    }
                  }
                },
                {
                  "term": {
                    "message": "ppap"
                  }
                }
              ],
              "must_not": [],
              "should": []
            }
          },
          "from": 0,
          "size": 10
        }
)
for tweet in response['hits']['hits']:
   print tweet['_source']['user']
   print tweet['_source']['message']
   print tweet['_source']['@timestamp']
   print "\n"


EveMoonshadow
RT @Mokuri3: ピコ太郎のPPAPが頭から抜けな過ぎてスプスのPPAP
https://t.co/wvmRfQoMJ0　←一応PPAP

下書き雑すぎてヤーン絵がズレてる～とか思ったけど、よくよく考えたら通常仕様で全く問題なかった（上達が見られな… https://t.co/MbFn7MRyHt
2016-10-02T15:46:02.000Z


satsuki_seimei
RT @Mokuri3: ピコ太郎のPPAPが頭から抜けな過ぎてスプスのPPAP
https://t.co/wvmRfQoMJ0　←一応PPAP

下書き雑すぎてヤーン絵がズレてる～とか思ったけど、よくよく考えたら通常仕様で全く問題なかった（上達が見られな… https://t.co/MbFn7MRyHt
2016-10-02T15:47:49.000Z


KellyKyw1010
YiYi ...PPAP.....Go ! XDDDDD #maltese #yiyi #puppy #cute #ppap #PPAP #dog #mommyloveu #puppylove #apple #pen… https://t.co/JXvqLXDa0k
2016-10-02T15:49:07.000Z


kiichiiiii1110
RT @ShonanMizuno: 2ヶ月記念日も仲良しこよし😊😊
ノリでPPAPやった＼(^o^)／
これからもよろしくねあやち😚

#PPAP 
#ペンパイナッポーアッポーペン https://t.co/sBLf2f8dZA
2016-10-02T15:39:23.000Z


NabillahLuthfia
RT @Daily_Namjoon: [FMV] PPAP - BTS VERSION
😂😂😂😂😂😂😂😂😂😂😂😂
©hyukning
#PPAP #BTS https://t.co/I1gH1LyWMa
2016-10-02T15:39:39.000Z


df6g_i
RT @17_min_seun: これヤバイｗｗｗｗｗｗｗｗｗｗ
PPAPブソクスンverｗｗｗｗｗｗ
クセになる本国ペンさん強い😂😂
#PPAP 
#笑ってはいけないSEVENTEEN24時 https://t.co/pGl6TfyWqp
2016-

In [25]:
response = es.search(
    index="logstash*",
    body={
          "query": {
            "term": {
              "message": "ppap"
            }
          },
          "aggs": {
            "user": {
              "terms": {
                "field": "user.raw",
                "size": 5
              }
            }
          }
        }
)

#for user in response['aggregations']['user']['buckets']:
#   print str(user['doc_count']) + "\t" + user['key'] 

21	nurqistinashah
6	Palmm_ExoiKON
4	Sinobu13
4	kkaixi
3	FmsgGgl
